In [1]:
%load_ext autoreload
%autoreload 2

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import StandardScaler
from keras.models import Model
from keras.layers import *    
import os 
import pyreadr
import numpy as np
import pyreadr
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed
import butterfly.album
import butterfly.Models
from itertools import combinations 
from joblib import parallel_backend
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm
from sklearn.metrics import r2_score
import pickle
from sklearn.model_selection import GroupKFold
from random import sample
from scipy import stats
from sklearn.metrics import mean_absolute_error
from collections import defaultdict
import time

Using TensorFlow backend.
/home/mxenoc/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mxenoc/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mxenoc/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/mxenoc/anaconda3/lib/python3.7/site-packages/tenso

In [58]:
#Model
folds = 10 #number of folds
features = 1 #number of features to predict
epochs = 180 #number of epochs
optimiser = 'adam' #model optimiser
loss = 'mse' #model loss
ntrees = 100

In [59]:
#Import your data
#DF = pyreadr.read_r('/Users/mxenoc/Desktop/workspace/butterfly/data/omics.RData')
DF = pyreadr.read_r('/home/mxenoc/workspace/butterfly/data/omics.RData')
DF = DF["DF"]

In [60]:
#Define the predictor datasets
omics = ['rna', 'plasma_l', 'serum_l', 'microb', 'immune', 'metabol', 'plasma_s']

In [61]:
os.chdir('/home/mxenoc/workspace/butterfly/src/butterfly')

In [62]:
with open('albums_all.pkl', 'rb') as f:
    albums_all = pickle.load(f)

In [63]:
with open('RF_predictor.pkl', 'rb') as f:
    RF_predictor = pickle.load(f)

In [64]:
with open('albums.pkl', 'rb') as f:
    albums = pickle.load(f)

In [65]:
os.chdir('/home/mxenoc/workspace/butterfly/data/pickles')

In [66]:
with open('groups_c.pkl', 'rb') as f:
    groups_c = pickle.load(f)

In [67]:
with open('responses_10.pkl', 'rb') as f:
    responses = pickle.load(f)
n_samples = 10 #same as the number of responses

In [ ]:
import numpy as np
from sklearn.model_selection import GroupKFold

In [ ]:
from sklearn.preprocessing import QuantileTransformer
feat_n = 0
groups = DF['patientID']

group_cols = groups_c[predictor_index]

#Get your response dataset
response = responses[predictor_index]
response_df = DF[response]
y = response_df.values
y = QuantileTransformer().fit_transform(y)

y = y[:,feat_n]

X = RF_predictor[predictor_index]

In [ ]:
results = []

y_prediction_train = []
y_observed_train = []

y_prediction_test = []
y_observed_test = []

group_kfold = GroupKFold(n_splits=folds)

In [ ]:
for train_index, test_index in group_kfold.split(X, y, groups):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

In [ ]:
import numpy as np
from sklearn.model_selection import GroupKFold

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)
GroupKFold(n_splits=2)

for train_index, test_index in group_kfold.split(X, y, groups):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]
     print(X_train, X_test, y_train, y_test)

In [ ]:
groups

In [ ]:
RF = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):

    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.LRF)(DF, responses, predictor_index, feat_n, 
                            RF_predictor, folds, ntrees, 'Lasso', 'matrix', groups_c, True)
                            for feat_n in range(0, n_samples)))
        
    RF['prediction_train'].append(prediction_train)
    RF['observed_train'].append(observed_train)
    RF['prediction_test'].append(prediction_test)
    RF['observed_test'].append(observed_test)

In [80]:
from sklearn.preprocessing import QuantileTransformer
feat_n = 0
groups = DF['patientID']

group_cols = groups_c[predictor_index]

#Get your response dataset
response = responses[predictor_index]
response_df = DF[response]
y = response_df.values
y = QuantileTransformer().fit_transform(y)

y = y[:,feat_n]

X = RF_predictor[predictor_index]

/home/mxenoc/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:2239: UserWarning: n_quantiles (1000) is greater than the total number of samples (68). n_quantiles is set to n_samples.
  % (self.n_quantiles, n_samples))


In [81]:
results = []

y_prediction_train = []
y_observed_train = []

y_prediction_test = []
y_observed_test = []

group_kfold = GroupKFold(n_splits=folds)

In [83]:
for train_index, test_index in group_kfold.split(X, y, groups):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

KeyError: "None of [Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 17, 18,\n            19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37,\n            38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56,\n            57, 58, 59, 61, 62, 63, 64, 65, 66],\n           dtype='int64')] are in the [columns]"

In [5]:
import numpy as np
from sklearn.model_selection import GroupKFold

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)
GroupKFold(n_splits=2)

for train_index, test_index in group_kfold.split(X, y, groups):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = X[train_index], X[test_index]
     y_train, y_test = y[train_index], y[test_index]
     print(X_train, X_test, y_train, y_test)

TRAIN: [0 1] TEST: [2 3]
[[1 2]
 [3 4]] [[5 6]
 [7 8]] [1 2] [3 4]
TRAIN: [2 3] TEST: [0 1]
[[5 6]
 [7 8]] [[1 2]
 [3 4]] [3 4] [1 2]


In [88]:
groups

array([0, 0, 2, 2])

In [34]:
RF = defaultdict(list)

for predictor_index in tqdm(range(len(omics))):

    prediction_train, observed_train, prediction_test, observed_test = zip(*Parallel(n_jobs=n_samples)
                           (delayed(butterfly.Models.LRF)(DF, responses, predictor_index, feat_n, 
                            RF_predictor, folds, ntrees, 'Lasso', 'matrix', groups_c, True)
                            for feat_n in range(0, n_samples)))
        
    RF['prediction_train'].append(prediction_train)
    RF['observed_train'].append(observed_train)
    RF['prediction_test'].append(prediction_test)
    RF['observed_test'].append(observed_test)

  0%|          | 0/7 [00:03<?, ?it/s]


KeyError: "None of [Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 17, 18,\n            19, 20, 21, 22, 23, 24, 25, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37,\n            38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56,\n            57, 58, 59, 61, 62, 63, 64, 65, 66],\n           dtype='int64')] are in the [columns]"